### Journée X/Stra-Dev Intégration continue

21 novembre 2017  

--- 

# L'intégration continue avec GitLab

![image.png](tanuki.png)

## *Session pratique*

---

Matthieu Boileau - <matthieu.boileau@math.unistra.fr>

*Contenu sous licence [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0)*

# GitLab CI

GitLab CI propose une chaîne complète d'intégration continue intégrée à son système de forge logicielle.

![](cicd_pipeline_infograph.png)
<span style="font-size:0.7em;"><https://docs.gitlab.com/ee/ci/README.html></span>

**Les avantages :**

- la simplicité de mise en oeuvre
- la robustesse d'une solution intégrée

## Le gitlab runner

### Introduction

- `gitlab-runner` est un démon installé sur une machine destinée à exécuter les tâches d'intégration continue.
- Pour chaque *runner* enregistré, `gitlab-runner` écoute l'instance de gitlab correspondante dans l'attente d'une tâche
- Cette tâche est déclenchée par des évènements (`push`, `merge`, etc.) sur le projet avec lequel il est lié
- Ce lien est créé à l'enregistrement du *runner* par un token associé au projet.

[**Doc en ligne**](https://docs.gitlab.com/ce/ci/runners/#shared-vs-specific-runners)

### Runners : *shared* vs *specific*

- *shared* : n'importe quel projet peut s'en servir. Seul l'administrateur du GitLab peut enregistrer des runners partagés.
- *specific* : seul le créateur du runner peut s'en servir dans d'autres projets

> **Attention** : les runners partagés sont pratiques mais peuvent présenter des [problèmes de sécurité](https://docs.gitlab.com/ee/ci/runners/README.html#be-careful-with-sensitive-information).


### Runners : *locked* vs *unlocked runners*

- *locked* : le runner ne peut être utilisé que pour le projet avec lequel il est lié
- *unlocked* : le créateur du runner peut l'utiliser dans ses autres projets

### Installer `gitlab-runner`

La **[procédure](https://docs.gitlab.com/runner/install/)** dépend du système d'exploitation.

## Fork du TP

Faire un *fork* du projet sur votre compte GitLab en allant sur la page d'accueil du projet

![fork.png](fork.png)

ou en suivant [ce lien](https://git.unistra.fr/m.boileau/tp-gitlab-ci/forks/new).

## Clone du TP

- Pour paramétrer les commandes de ce notebook, positionner les variables d'environnement suivantes :

In [ ]:
export USERNAME="m.boileau"  # votre login sur git.unistra.fr
export TPDIR=$HOME         # le répertoire où vous souhaitez installer le TP

- Cloner localement votre projet

In [ ]:
cd $TPDIR
git clone git@git.unistra.fr:$USERNAME/tp-gitlab-ci.git

## Enregistrer  un nouveau runner

> Si `gitlab-runner` n'est pas installé sur votre machine, rdv à la section **Utiliser un runner partagé**

Engistrer un runner consiste à créer un nouveau runner associé à un projet.
Si vous n'êtes pas administrateur du GitLab, il s'agit forcément d'un runner spécifique. Celui-ci se trouve dans 
votre projet, rubrique :

```
Settings > CI / CD > Runners settings
```

Dans le terminal, maintenant :

In [ ]:
export CI_SERVER_URL=https://git.unistra.fr/  # Do not change for this session
export RUNNER_NAME=a_runner_for_$USERNAME
export REGISTRATION_TOKEN=xxx  # Replace by the value provided by the runner setting page
export REGISTER_NON_INTERACTIVE=true
#gitlab-runner register --executor shell        # sous MacOS
#sudo gitlab-runner register --executor shell   # sous Linux

Ou bien lancez la commande en mode interactif et en suivant pas à pas la [procédure](http://docs.gitlab.com/runner/register/#registering-runners):
```
gitlab-runner register       # sous MacOS
sudo gitlab-runner register  # sous Linux
```

- Retournez sur la page de paramètrage des runners
- Repérez votre nouveau runner
- Editez-le et ajouter les tags `specific, shell`

Votre runner est maintenant prêt à travailler. Nous allons le tester en lui soumettant une tâche GitLab CI déclenchée par un `git push` sur votre projet.

## Exo1 : helloworld en bash

### Mettre en place une action d'intégration continue

Aller dans le répertoire racine du clone local de votre projet et basculer sur la branche `exo1`

In [ ]:
cd $TPDIR
cd tp-gitlab-ci

In [ ]:
git pull
git checkout exo1

Repérer le fichier `.gitlab-ci.yml`

In [ ]:
ls -al

L'éditer avec l'éditeur de votre choix

In [ ]:
cat .gitlab-ci.yml

- `helloworld` indique le nom du job d'intégration continue
- `tags` permet de sélectionner des runners sur la base de mot-clés
- `scripts` correspond aux lignes de commande `bash` que vous souhaitez exécuter

Modifiez la ligne de script, par exemple:

In [ ]:
echo '    - echo "bonjour, monde"' >> .gitlab-ci.yml
cat .gitlab-ci.yml

Enregistrez avec git

In [ ]:
git add .gitlab-ci.yml
git commit -m "Update .gitlab-ci.yml"

Pousser vos modifications

In [ ]:
git push

Sur GitLab, observez l'exécution de votre job dans la rubrique de votre projet :
```
CI / CD > Pipelines
```

### Provoquer une erreur

Dans `.gitlab-ci.yml`, introduisez une erreur dans une des lignes du script et observer l'effet dans `CI / CD > Pipelines` après avoir pousser les modifications.

> **Note :** les fichiers peuvent s'éditer directement dans GitLab.